In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Part_Number'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=' + _ for _ in list_url]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = json.loads(requests.get(url,
                                               headers=UA.get_User_Agent_Requests(),
                                               proxies=Proxy.get_Proxy_Requests()).text)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for _ in resp['pp']['buyersGuides']:
                    list_vehicle.append(_['buyersGuideDesc'].strip())

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': resp['pd']['basePart'].strip(),
                                         'Part_Type': resp['pd']['partDesc_en'].strip(),
                                         'Part_Type_Long': resp['pd']['partLongDesc_en'].strip(),
                                         'Brand': resp['pd']['brand'].strip(),
                                         'Pop': resp['pd']['pop'].strip(),
                                         'Per_Car': resp['pd']['qtyEach'],
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': 'https://ecatalog.smpcorp.com/V2/STD/#/partdetail/part/' + resp['pd']['basePart'].strip(),
                                         'Src': ''}])
                
                for name_value in resp['pp']['partSpecs']:
                    df_temp[name_value['attributeName_en'].strip().lower()] = name_value['siteValue_en'].strip()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：444

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR106  <->  [ok] - 剩余数量：439
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR101  <->  [ok] - 剩余数量：438
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR103  <->  [ok] - 剩余数量：437
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR108  <->  [ok] - 剩余数量：436
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR105  <->  [ok] - 剩余数量：435
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR112  <->  [ok] - 剩余数量：434
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR102  <->  [ok] - 剩余数量：433
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR115  <->  [ok] - 剩余数量：432
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR107  <->  [ok] - 剩余数量：431
https://ecatalog.smpcorp.com/V2/STD/api/part/p

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR220  <->  [ok] - 剩余数量：360
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR223  <->  [ok] - 剩余数量：359
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR222  <->  [ok] - 剩余数量：358
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR226  <->  [ok] - 剩余数量：357
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR225  <->  [ok] - 剩余数量：356
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR230  <->  [ok] - 剩余数量：355
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR212  <->  [ok] - 剩余数量：354
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR194  <->  [ok] - 剩余数量：353
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR231  <->  [ok] - 剩余数量：352
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4266  <->  [ok] - 剩余数量：282
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4272  <->  [ok] - 剩余数量：281
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4273  <->  [ok] - 剩余数量：280
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4290  <->  [ok] - 剩余数量：279
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4293  <->  [ok] - 剩余数量：278
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4278  <->  [ok] - 剩余数量：277
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4295  <->  [ok] - 剩余数量：276
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4296  <->  [ok] - 剩余数量：275
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4291  <->  [ok] - 剩余数量：274
https://ecatalog.smpcorp.com/V2/STD/api/part/p

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4372  <->  [ok] - 剩余数量：204
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4376  <->  [ok] - 剩余数量：203
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4379  <->  [ok] - 剩余数量：202
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4378  <->  [ok] - 剩余数量：201
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4381  <->  [ok] - 剩余数量：200
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4380  <->  [ok] - 剩余数量：199
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4384  <->  [ok] - 剩余数量：198
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4375  <->  [ok] - 剩余数量：197
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4386  <->  [ok] - 剩余数量：196
https://ecatalog.smpcorp.com/V2/STD/api/part/p

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4491  <->  [ok] - 剩余数量：126
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4492  <->  [ok] - 剩余数量：125
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4466  <->  [ok] - 剩余数量：124
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4510  <->  [ok] - 剩余数量：123
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4479  <->  [ok] - 剩余数量：122
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4456  <->  [ok] - 剩余数量：121
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4507  <->  [ok] - 剩余数量：120
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4512  <->  [ok] - 剩余数量：119
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4496  <->  [ok] - 剩余数量：118
https://ecatalog.smpcorp.com/V2/STD/api/part/p

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=G04028  <->  [ok] - 剩余数量：47
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=G04027  <->  [ok] - 剩余数量：46
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=G04029  <->  [ok] - 剩余数量：45
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=G17003  <->  [ok] - 剩余数量：44
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=G04005  <->  [ok] - 剩余数量：43
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=G17006  <->  [ok] - 剩余数量：42
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=G17005  <->  [ok] - 剩余数量：41
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=J04001  <->  [ok] - 剩余数量：40
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=I04004  <->  [ok] - 剩余数量：39
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PAR